In [6]:
import pandas as pd
from datetime import time

data = pd.read_csv('C:/Users/Nifemi/Code/vsc/TenzoInt/work_shifts.csv')

In [7]:
data

,break_notes,end_time,pay_rate,start_time
0,15-18,23:00,10.0,10:00
1,18.30-19.00,23:00,12.0,18:00
2,4PM-5PM,22:30,14.0,12:00
3,3-4,18:00,10.0,09:00
4,4-4.10PM,23:00,20.0,09:00
5,15 - 17,23:00,10.0,11:00
6,11 - 13,16:00,10.0,10:00


In [8]:
#Formulate the dictionary with the times as key and total as value

startTimes = lambda x: time(hour = int(x.split(":")[0]), minute = int(x.split(":")[1]))

earliestShiftStart = data.start_time.apply(startTimes).min()

latestShiftEnd = data.end_time.apply(startTimes).max()

allWorkingHoursList = list(range(earliestShiftStart.hour,latestShiftEnd.hour,1))

allWorkingHoursList = list(map(lambda x : f"{x}", allWorkingHoursList))

labourCostDict = dict.fromkeys(allWorkingHoursList, 0)


In [9]:
def func(x):
    if "PM" in x:
        x = x.replace("PM", "")
        splitTime = x.split("-")
        if "." in x:
            start = int(splitTime[0]) + 12
            end = splitTime[1].replace(splitTime[1][0], f"{int(splitTime[1][0]) + 12}") 
        else:
            start = int(splitTime[0]) + 12
            end = int(splitTime[1]) + 12
    else:
        splitTime = x.split("-")
        if float(splitTime[0]) < 9:
            start = int(splitTime[0]) + 12
            end = int(splitTime[1]) + 12
        else:
            start = splitTime[0].strip()
            end = splitTime[1].strip()
    return [start, end]

breakTimes = list(map(func, data.break_notes))

In [10]:
startTimes = []
endTimes = []

for breaks in breakTimes:
    startTimes.append(breaks[0])
    endTimes.append(breaks[1])

print(startTimes)
print(endTimes)

['15', '18.30', 16, 15, 16, '15', '11']
['18', '19.00', 17, 16, '16.10', '17', '13']


In [11]:
data["break_start"] = startTimes
data["break_end"] = endTimes

data

,break_notes,end_time,pay_rate,start_time,break_start,break_end
0,15-18,23:00,10.0,10:00,15,18
1,18.30-19.00,23:00,12.0,18:00,18.30,19.00
2,4PM-5PM,22:30,14.0,12:00,16,17
3,3-4,18:00,10.0,09:00,15,16
4,4-4.10PM,23:00,20.0,09:00,16,16.10
5,15 - 17,23:00,10.0,11:00,15,17
6,11 - 13,16:00,10.0,10:00,11,13


In [12]:
def convertToTimeObject(x):
    if type(x) != int and ":" in x:
        splitTime = x.split(":")
        times = time(hour=int(splitTime[0]), minute=int(splitTime[1]))
    elif type(x) != int and "." in x:
        splitTime = x.split(".")
        times = time(hour=int(splitTime[0]), minute=int(splitTime[1]))
    else:
        times = time(hour=int(x))

    return times


In [13]:
data["end_time"] = list(map(convertToTimeObject, data.end_time))
data["start_time"] = list(map(convertToTimeObject, data.start_time))
data["break_start"] = list(map(convertToTimeObject, data.break_start))
data["break_end"] = list(map(convertToTimeObject, data.break_end))


data

,break_notes,end_time,pay_rate,start_time,break_start,break_end
0,15-18,23:00:00,10.0,10:00:00,15:00:00,18:00:00
1,18.30-19.00,23:00:00,12.0,18:00:00,18:30:00,19:00:00
2,4PM-5PM,22:30:00,14.0,12:00:00,16:00:00,17:00:00
3,3-4,18:00:00,10.0,09:00:00,15:00:00,16:00:00
4,4-4.10PM,23:00:00,20.0,09:00:00,16:00:00,16:10:00
5,15 - 17,23:00:00,10.0,11:00:00,15:00:00,17:00:00
6,11 - 13,16:00:00,10.0,10:00:00,11:00:00,13:00:00


In [14]:
for key, value in labourCostDict.items():
    hourBeginning = time(hour=int(key))

    counter=0
    for index, row in data.iterrows():
        if row['start_time'] <= hourBeginning < row['end_time'] and (row['break_end'] <= hourBeginning or hourBeginning < row['break_start']):
            if hourBeginning.hour == row['end_time'].hour:
                counter = counter + (row['pay_rate'] * (60 - row['end_time'].minute)/60)
            else:
                counter = counter + row['pay_rate']
    labourCostDict[key] = counter

labourCostDict



{'9': 30.0,
 '10': 50.0,
 '11': 50.0,
 '12': 64.0,
 '13': 74.0,
 '14': 74.0,
 '15': 44.0,
 '16': 10.0,
 '17': 54.0,
 '18': 66.0,
 '19': 66.0,
 '20': 66.0,
 '21': 66.0,
 '22': 59.0}

In [15]:
# from datetime import time

# times = time(hour=10) 

# timestew = time(hour=10, minute=30)

# print(bool(times > timestew))

#We can deal with the times by splitting them on the colon and creating a time object out of the value
#This will allow is to compare times
#All our times should be time objects to allow for comparisons
#For break times we need to also create time objects, need to convert to 24h time if "pm" comes after the number
#Need to reformat 